# Image Classification Program 

#### Import all necessary packages before starting the program. This time usng Pytorch and Tf more.

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image

import tensorflow as tf
import torch

#import datasets and transform function
import torchvision 
import torchvision.transforms as transforms

import torch.nn as nn
import torch.optim as optim

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#### Load and Normalize Data

In [2]:
#define transform to convert images into Tensors then normalize them
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ]
)

#set batch size
batch_size = 4
#set number of workers
num_workers = 2

#load training data
trainset = torchvision.datasets.FashionMNIST(root='./data',train=True, download=True,
                                        transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, 
                                        shuffle=True, num_workers=num_workers)

#load test data
testset = torchvision.datasets.FashionMNIST(root='./data',train=False, download=True,
                                        transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=num_workers)


#### Set up a convolutional neural network model

In [3]:

class Net(nn.Module):
    
    def __init__(self):
        '''init the neural network'''
        super(Net, self).__init__()
        
        #set up 3x3 convolution kernels
        self.conv1 = nn.Conv2d(1, 6, 3)
        #set up 2x2 max pooling window
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, mod):
        '''defines the forward propogation algorithm'''
        #apply MaxPooling on the relu'd results from conv1/con2
        mod = self.pool(nn.functional.relu(self.conv1(mod)))
        mod = self.pool(nn.functional.relu(self.conv2(mod)))
        mod = mod.flatten(1)
        #run fc1 and fc2 layers through relu
        mod = nn.functional.relu(self.fc1(mod))
        mod = nn.functional.relu(self.fc2(mod))
        #fc3 will give us the final 10 layers
        mod = self.fc3(mod)
        return mod
    
net = Net()
print(net)
        
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### Define the loss func and optimzer

In [4]:
#similar to Keras's sparse categoricla cross-entropy 
crit = nn.CrossEntropyLoss()
#I used Adam in other program, just wanted to try sgd instead this time
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#### Train the nn

In [5]:
print(torch.cuda.is_available())

for epoch in range(5):
    r_loss = 0.0
    for i, data in enumerate(trainloader,0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = crit(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #print stats
        r_loss += loss.item()
        if i % 2000 == 1999: #print every 2000 iters
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, r_loss/2000))
            r_loss = 0.0


print('Finished Training')

False
[1,  2000] loss: 1.318
[1,  4000] loss: 0.647
[1,  6000] loss: 0.562
[1,  8000] loss: 0.524
[1, 10000] loss: 0.483
[1, 12000] loss: 0.451
[1, 14000] loss: 0.443
[2,  2000] loss: 0.408
[2,  4000] loss: 0.391
[2,  6000] loss: 0.374
[2,  8000] loss: 0.383
[2, 10000] loss: 0.361
[2, 12000] loss: 0.373
[2, 14000] loss: 0.347
[3,  2000] loss: 0.329
[3,  4000] loss: 0.349
[3,  6000] loss: 0.326
[3,  8000] loss: 0.328
[3, 10000] loss: 0.325
[3, 12000] loss: 0.315
[3, 14000] loss: 0.316
[4,  2000] loss: 0.301
[4,  4000] loss: 0.305
[4,  6000] loss: 0.302
[4,  8000] loss: 0.283
[4, 10000] loss: 0.298
[4, 12000] loss: 0.303
[4, 14000] loss: 0.285
[5,  2000] loss: 0.273
[5,  4000] loss: 0.276
[5,  6000] loss: 0.281
[5,  8000] loss: 0.269
[5, 10000] loss: 0.285
[5, 12000] loss: 0.262
[5, 14000] loss: 0.281
Finished Training


In [15]:
data_iter = iter(testloader)
images, labels = data_iter.next()

# print images

print('Actual: ', ' '.join('%d' % labels[j] for j in range(4)))

outputs = net(images)

_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%d' % predicted[j] for j in range(4)))

Actual:  9 2 1 1
Predicted:  9 2 1 1
